Neural Network Optimization #2

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

Mounted at /content/drive


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop non-beneficial columns
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis = 1)

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 50
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff].index.tolist()
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: count, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 10
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: count, dtype: int64

In [5]:
# application_df = application_df.drop(['APPLICATION_TYPE'],axis=1)
# Convert categorical data to numeric with `pd.get_dummies`
application_encoded_df = pd.get_dummies(application_df).astype(int)

In [6]:
# Split our preprocessed data into our features and target arrays
X = application_encoded_df.drop(['IS_SUCCESSFUL'], axis=1)
y = application_encoded_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=400, activation="relu", input_dim=X_train_scaled.shape[-1]))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=200, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=200, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 400)               26800     
                                                                 
 dense_1 (Dense)             (None, 200)               80200     
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 147401 (575.79 KB)
Trainable params: 147401 (575.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Create a callback that saves the model's weights every five epochs
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = 'model_weights_epoch_{epoch:02d}.h5'

checkpoint_callback = ModelCheckpoint(filepath, save_weights_only = True, save_freq = 5)

In [11]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=50, callbacks = [checkpoint_callback])

Epoch 1/50
804/804 [==============================] - 8s 7ms/step - loss: 0.5666 - accuracy: 0.7226
Epoch 2/50
804/804 [==============================] - 8s 9ms/step - loss: 0.5537 - accuracy: 0.7296
Epoch 3/50
804/804 [==============================] - 7s 8ms/step - loss: 0.5498 - accuracy: 0.7308
Epoch 4/50
804/804 [==============================] - 7s 9ms/step - loss: 0.5483 - accuracy: 0.7333
Epoch 5/50
804/804 [==============================] - 7s 9ms/step - loss: 0.5459 - accuracy: 0.7342
Epoch 6/50
804/804 [==============================] - 5s 7ms/step - loss: 0.5455 - accuracy: 0.7348
Epoch 7/50
804/804 [==============================] - 8s 10ms/step - loss: 0.5444 - accuracy: 0.7359
Epoch 8/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5431 - accuracy: 0.7355
Epoch 9/50
804/804 [==============================] - 8s 10ms/step - loss: 0.5429 - accuracy: 0.7357
Epoch 10/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5423 - accuracy: 0.73

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5842 - accuracy: 0.7292 - 834ms/epoch - 3ms/step
Loss: 0.5841538310050964, Accuracy: 0.7292128205299377


In [15]:
# Export our model to HDF5 file
nn2.save("/content/drive/MyDrive/Colab Notebooks/AlphabetSoupCharity_Optimization2.h5")